<a href="https://colab.research.google.com/github/purplepatch-everyday/RecSys/blob/main/Simple_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install wget
import wget
import zipfile
import os
import pandas as pd


In [54]:
# Google colab mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###If you havn't downloaded movielens data yet, uncomment the code below to download and extract files

In [55]:
# Download data
# url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# wget.download(url,'/content/drive/MyDrive/data')


In [56]:
# Extract zipfile
# you can change to your own path 

# with zipfile.ZipFile("/content/drive/MyDrive/data/ml-latest-small.zip","r") as zip_ref:
#   zip_ref.extractall("/content/drive/MyDrive/data")

In [57]:
path = "/content/drive/MyDrive/data/ml-latest-small/"
os.listdir(path)

['movies.csv', 'README.txt', 'links.csv', 'ratings.csv', 'tags.csv']

In [58]:
# Read movies,ratings,tags files

ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [59]:
# Check ratings_df, movies_df,tags_df

print(ratings_df.head())
print(ratings_df.shape)

print(movies_df.head())
print(movies_df.shape)

print(tags_df.head())
print(tags_df.shape)

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
(100836, 4)
                                      title                                       genres
movieId                                                                                 
1                          Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
2                            Jumanji (1995)                   Adventure|Children|Fantasy
3                   Grumpier Old Men (1995)                               Comedy|Romance
4                  Waiting to Exhale (1995)                         Comedy|Drama|Romance
5        Father of the Bride Part II (1995)                                       Comedy
(9742, 2)
   userId  movieId              tag   timestamp
0       2    60756            funny  1445714994
1       2    60756  Highly quot

In [60]:
# Find unique number of userId, movieId

print("# of unique userId:", len(ratings_df["userId"].unique()))
print("# of unique movieId:", len(ratings_df["movieId"].unique()))

# of unique userId: 610
# of unique movieId: 9724


In [61]:
# Create pivot table (row = movieId, col= userId, value = rating) from ratings_df

ratings_pivot = pd.pivot_table(ratings_df,index="movieId",columns="userId",values="rating").fillna(0)

print(ratings_pivot.head())

userId   1    2    3    4    5    6    7    ...  604  605  606  607  608  609  610
movieId                                     ...                                   
1        4.0  0.0  0.0  0.0  4.0  0.0  4.5  ...  3.0  4.0  2.5  4.0  2.5  3.0  5.0
2        0.0  0.0  0.0  0.0  0.0  4.0  0.0  ...  5.0  3.5  0.0  0.0  2.0  0.0  0.0
3        4.0  0.0  0.0  0.0  0.0  5.0  0.0  ...  0.0  0.0  0.0  0.0  2.0  0.0  0.0
4        0.0  0.0  0.0  0.0  0.0  3.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5        0.0  0.0  0.0  0.0  0.0  5.0  0.0  ...  3.0  0.0  0.0  0.0  0.0  0.0  0.0

[5 rows x 610 columns]


In [62]:
# Convert dataframe to sparse matrix
from scipy.sparse import csr_matrix

ratings_pivot_sparse = csr_matrix(ratings_pivot.values)
print(ratings_pivot_sparse)

  (0, 0)	4.0
  (0, 4)	4.0
  (0, 6)	4.5
  (0, 14)	2.5
  (0, 16)	4.5
  (0, 17)	3.5
  (0, 18)	4.0
  (0, 20)	3.5
  (0, 26)	3.0
  (0, 30)	5.0
  (0, 31)	3.0
  (0, 32)	3.0
  (0, 39)	5.0
  (0, 42)	5.0
  (0, 43)	3.0
  (0, 44)	4.0
  (0, 45)	5.0
  (0, 49)	3.0
  (0, 53)	3.0
  (0, 56)	5.0
  (0, 62)	5.0
  (0, 63)	4.0
  (0, 65)	4.0
  (0, 67)	2.5
  (0, 70)	5.0
  :	:
  (9700, 337)	2.5
  (9701, 337)	3.0
  (9702, 183)	4.0
  (9702, 247)	3.5
  (9703, 317)	2.5
  (9704, 209)	1.0
  (9705, 461)	2.5
  (9706, 49)	3.5
  (9707, 337)	1.5
  (9708, 337)	4.0
  (9709, 337)	1.0
  (9710, 337)	1.5
  (9711, 337)	1.0
  (9712, 337)	1.0
  (9713, 183)	4.5
  (9714, 183)	3.5
  (9715, 183)	3.0
  (9716, 183)	4.0
  (9717, 183)	4.0
  (9718, 183)	3.5
  (9719, 183)	4.0
  (9720, 183)	3.5
  (9721, 183)	3.5
  (9722, 183)	3.5
  (9723, 330)	4.0


In [63]:
# # Split Train, Test dataset 
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(ratings_df,test_size =0.2, random_state = 1004)

# print("shape of original dataset:", ratings_df.shape)
# print("shape of train dataset:",train.shape)
# print("shape of test dataset:",test.shape)


##Simple Recommender System 

1.   Rating prediction based on average movie ratings
2.   Rating prediction based on average user ratings

###Prediction Errors

1.   Root Mean Square Error (RMSE) 





In [64]:
print(ratings_df.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


##1. Rating prediction with average movie ratings

In [65]:
# Create df with average movie ratings by movie id
movie_rating_df ["Pred_by_movie"] = ratings_df.groupby("movieId").mean()["rating"]
movie_rating_df= movie_rating_df["Pred_by_movie"]
movie_rating_df = movie_rating_df.to_frame(name="Pred_by_movie")
print(movie_rating_df.head())
print(movie_rating_df.shape)

         Pred_by_movie
movieId               
1             3.920930
2             3.431818
3             3.259615
4             2.357143
5             3.071429
(9724, 1)


In [66]:
# merge movie_rating_df and ratings_df (left join)
movie_average_final_df = pd.merge(left=ratings_df, right = movie_rating_df, how = "left",left_on="movieId",right_on="movieId")
movie_average_final_df.head()


,userId,movieId,rating,timestamp,Pred_by_movie
0,1,1,4.0,964982703,3.920930
1,1,3,4.0,964981247,3.259615
2,1,6,4.0,964982224,3.946078
3,1,47,5.0,964983815,3.975369
4,1,50,5.0,964982931,4.237745


In [67]:
# Prediction errors (RMSE)
import numpy as np
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_true=movie_average_final_df["rating"].values, y_pred=movie_average_final_df["Pred_by_movie"].values)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 0.8761725163561306


##2. Rating prediction based on average user ratings

In [72]:
print(ratings_df.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [82]:
# Create df with average movie ratings by user id

user_rating_df["Pred_by_user"] = ratings_df.groupby("userId").mean()["rating"]
user_rating_df = user_rating_df["Pred_by_user"]
user_rating_df = user_rating_df.to_frame(name="Pred_by_user")
print(user_rating_df.head())

        Pred_by_user
userId              
1           4.366379
2           3.948276
3           2.435897
4           3.555556
5           3.636364


In [84]:
# merge user_rating_df and movie_average_final_df (left join)
user_average_final_df = pd.merge(left=movie_average_final_df, right = user_rating_df, how = "left",left_on="userId",right_on="userId")
user_average_final_df.head()

,userId,movieId,rating,timestamp,Pred_by_movie,Pred_by_user
0,1,1,4.0,964982703,3.920930,4.366379
1,1,3,4.0,964981247,3.259615,4.366379
2,1,6,4.0,964982224,3.946078,4.366379
3,1,47,5.0,964983815,3.975369,4.366379
4,1,50,5.0,964982931,4.237745,4.366379


In [85]:
# Prediction errors (RMSE)

mse = mean_squared_error(y_true=user_average_final_df["rating"].values, y_pred=user_average_final_df["Pred_by_user"].values)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 0.934721133650792
